In [ ]:
import os
import numpy as np
import dill
import pickle
from bottleneck import push
from fancyimpute import IterativeImputer
from miss_utils import *

In [ ]:
data_path = 'extracts/'
with open(os.path.join(data_path, '3d_tensor_set_a.pkl'), 'rb') as f:
    origional_a = pickle.load(f)
    
with open(os.path.join(data_path, '3d_tensor_set_b.pkl'), 'rb') as f:
    origional_b = pickle.load(f)    

with open(os.path.join(data_path, '3d_tensor_set_c.pkl'), 'rb') as f:
    origional_c = pickle.load(f)
    
origional = np.vstack([origional_a,origional_b,origional_c]) 
mask=~np.isnan(origional)*1

In [ ]:

mins = []
maxs = []
flatten= origional.reshape(origional.shape[0]*origional.shape[1], origional.shape[2])

for i in range(origional.shape[2]):

        mins.append(np.nanmin(flatten[:,i]))

        maxs.append(np.nanmax(flatten[:,i]))

normalized =normalize(origional,mins,maxs)

Patient level introduce

In [ ]:
seed = 1234
new_tensor, o,miss_indices=array_rsme(normalized,mask, 0.1, seed)
LO =push(new_tensor, axis=1)
LOCV= np.where(np.isnan(LO), 0, LO)
zero= np.where(np.isnan(new_tensor), 0, new_tensor)
means, stds = [],[]
flatten= new_tensor.reshape(origional.shape[0]*origional.shape[1], 35)

for i in range(35):
    means.append(np.nanmean(flatten[:,i]))
    stds.append(np.nanstd(flatten[:,i]))
mean_imputed =mean_fill(new_tensor, means)
mask_miss=~np.isnan(new_tensor)*1
imputed=prepare_fills(new_tensor, means)
IMM= create_individualized_missingness_mask(mask_miss)

In [ ]:
# with open('extracts/X_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(new_tensor, outfile, pickle.HIGHEST_PROTOCOL)  
# with open('extracts/new_miss_oo_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(zero, outfile, pickle.HIGHEST_PROTOCOL) 
# with open('extracts/new_miss_0.1_back_forward_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(imputed, outfile, pickle.HIGHEST_PROTOCOL) 
# with open('extracts/mask_miss_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(mask_miss, outfile, pickle.HIGHEST_PROTOCOL)  
# with open('extracts/IMM_0.1_'+str(seed)+'.pkl', 'wb') as outfile:
#     dill.dump(IMM, outfile, pickle.HIGHEST_PROTOCOL)  

In [ ]:
mice_impute = IterativeImputer()
flatten= new_tensor.reshape(new_tensor.shape[0]*new_tensor.shape[1], new_tensor.shape[2])
MICE = mice_impute.fit_transform(flatten)
MICE=MICE.reshape(new_tensor.shape[0], new_tensor.shape[1], new_tensor.shape[2])
MICE =MICE.copy()

In [ ]:
experiment_name = "IMM_population_1234_01"
saved_path = 'data/'
observed_only_real = np.load(saved_path + experiment_name + '.npz')["observed_only_real"]
observed_only_rec = np.load(saved_path + experiment_name + '.npz')["observed_only_rec"]
IMM_real = np.load(saved_path + experiment_name + '.npz')["IMM_real"]
IMM_rec = np.load(saved_path + experiment_name + '.npz')["IMM_rec"]
imputed_ours =(mask_miss * (zero)+ ((1-mask_miss) *IMM_rec))
GP = np.load("GP_0.1.npy")
GP_ =(mask_miss * (zero)+ ((1-mask_miss) *GP))

with open(os.path.join('SAITS_01_miss_ratio.pkl'), 'rb') as f:
           SAITS = pickle.load(f)    
with open(os.path.join('BRITS_01_miss_ratio.pkl'), 'rb') as f:
            BRITS = pickle.load(f)  

In [ ]:
imputed_ours =(mask_miss * (zero)+ ((1-mask_miss) *observed_only_rec))

In [ ]:
MICE =MICE.copy()

In [ ]:
flat= np.zeros((12000,48,35)).reshape(-1)
flat[miss_indices] = 1
indicating_mask=flat.reshape(12000,48,35)


In [ ]:
print("MAE")
print("----------------------")
get_patient_level_MAE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MAE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # calculate mean absolute error on the ground truth (artificially-missing values)

In [ ]:
print("RMSE")
print("----------------------")
get_patient_level_RMSE_array(LOCV, normalized, indicating_mask, "LOCF") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_RMSE_array(imputed_ours, normalized, indicating_mask, "IGNITE ") # cl

In [ ]:
print("MSE")
print("----------------------")
get_patient_level_MSE_array(LOCV, normalized, indicating_mask, "LOCF") 
get_patient_level_MSE_array(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_patient_level_MSE_array(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

Calculate Population level sum

In [ ]:
get_population_RMSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
get_population_MSE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE(imputed_ours, normalized, indicating_mask, "IGNITE") # c

In [ ]:
get_population_MAE(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

get weighted population sum

In [ ]:
get_population_MAE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MAE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)


In [ ]:
get_population_MAE_weighted(imputed_ours2, normalized, indicating_mask, "IGNITE lower")# calculate mean absolute error on the ground truth (artificially-missing values)


In [ ]:
get_population_MAE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MAE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
get_population_RMSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_RMSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_RMSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
get_population_MSE_weighted(LOCV, normalized, indicating_mask, "LOCF") 
get_population_MSE_weighted(mean_imputed, normalized, indicating_mask, "mean") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(MICE, normalized, indicating_mask, "mice") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(BRITS, normalized, indicating_mask, "BRITS")# calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(SAITS, normalized, indicating_mask, "SAITS") # calculate mean absolute error on the ground truth (artificially-missing values)
get_population_MSE_weighted(imputed_ours, normalized, indicating_mask, "IGNITE") # cl

In [ ]:
# def get_population_RMSE_weighted(imputation,observed, indicating_mask, name):
#     array_RMSE = []
#     array_weight =[]
#     observed = np.where(np.isnan(observed), 0, observed)

#     for count in range(imputation.shape[0]):
#          selected = indicating_mask[count]
#          w= (selected.sum())/(indicating_mask.sum())
#          array_weight.append(w)
#          if len(selected)> 0:
#            mse=np.sum(np.square(imputation[count] - observed[count]) * selected) / (np.sum(selected) + 1e-9)
#            rms=np.sqrt(mse)
#          else:
#              rms = 0.0
#          array_RMSE.append(rms*w)
#     print(name, np.sum(array_RMSE))

In [ ]:
imputed_ours_oo[10,:20,1]

In [ ]:
imputed_ours[10,:20,1]

In [ ]:
#introduced 
IMM[10,:20,1]

In [ ]:
normalized[10,:20,1]